In [35]:
from dataclasses import dataclass
from collections import defaultdict, namedtuple
from openpyxl import load_workbook

dmi = load_workbook("./fin/data_export_output_all_submissions_march-14_RC data management cleaned.xlsx")["FTE"]
#dmi_data_slice = dmi["E2:O247"]
dmi_data_slice = dmi["P2:Z290"]
iop = load_workbook("./fin/IOP.xlsx")["FUNDED TABLES"]
iop_slice = [row for row in iop if row[4].value == "x"]

@dataclass
class DMCycle:
    name : str
    dmi_index : int
    centralize : float
    iop_index : int = 0
    allocation : float = 0 
    dmi_fte_count : int = 0
    iop_personnel : float = 0
    iop_fte : float = 0
    iop_OandM : float = 0

    def totals(self):
        return f'''{self.name};{self.iop_fte:,.0f}; {self.iop_personnel:,.0f}; { self.iop_OandM:,.0f} '''


dmi_stages = [
    ("Collection",0.1), 
    ("Receipt",1),
    ("Management",1), 
    ("Cleaning",0.9), 
    ("Analysis",0.2), 
    ("Dissemination",0.4), 
    ("Other",0.15)
]

dmi_data = {
    k[0] : DMCycle(name=k[0],centralize=k[1],dmi_index=x+4,iop_index=x+8)
    for x,k in enumerate(dmi_stages)
}

total_dmi_FTE = sum( x[3].value for x in dmi_data_slice)/100
for x,row in enumerate(dmi_data_slice):
    system_allocation = row[3].value
    total_allocation = sum( row[x.dmi_index].value for x in dmi_data.values())
    if not system_allocation or not total_allocation:
        continue
    for step in dmi_data.values():
        try:
            step.dmi_fte_count += (row[step.dmi_index].value/total_allocation) * (system_allocation/100)
        except:
            print(x)

for step in dmi_data.values():
    step.allocation = step.dmi_fte_count / total_dmi_FTE
    print(f"{step.name};{step.allocation*100:.1f}%")

Collection;9.1%
Receipt;4.3%
Management;5.5%
Cleaning;6.6%
Analysis;24.9%
Dissemination;15.2%
Other;34.4%


In [23]:
other = dmi_data["Other"]
@dataclass
class Accumulator:
    fte : float =0
    sal : float = 0
    om : float = 0
iop_totals = Accumulator()
branches = defaultdict(lambda : Accumulator())
for x,row in enumerate(iop_slice):
    steps = [x for x in dmi_data.values() if row[x.iop_index].value == "x" and x.name != "other"]
    sal = row[14].value  or 0 
    fte= row[19].value  or 0 
    om =  row[15].value  or 0 
    branch = row[1].value
    match len(steps):
        case 6:
            for step in steps:
                step.iop_personnel += (sal * step.allocation * step.centralize)
                step.iop_fte += (fte* step.allocation * step.centralize) 
                step.iop_OandM += (om * step.allocation * step.centralize)
                
                branches[branch].sal += (sal * step.allocation * step.centralize)
                branches[branch].fte += (fte* step.allocation * step.centralize)                
                branches[branch].om += (om * step.allocation * step.centralize)
        case 1 | 2 | 3 | 4 | 5:
            rebase_total = sum(x.allocation for x in steps) + other.allocation
            for step in steps:
                step_allocation = step.allocation / rebase_total
                
                step.iop_personnel += (sal * step_allocation * step.centralize)
                step.iop_fte += (fte * step_allocation * step.centralize)
                step.iop_OandM += (om * step_allocation * step.centralize)

                branches[branch].sal += (sal * step_allocation * step.centralize)
                branches[branch].fte += (fte * step_allocation * step.centralize)
                branches[branch].om += (om * step_allocation * step.centralize)
        case 0:
            continue
    iop_totals.sal += sal
    iop_totals.fte += fte
    iop_totals.om += om
    
    other.iop_personnel += (sal * other.allocation * other.centralize)
    other.iop_fte += (fte * other.allocation * other.centralize)
    other.iop_OandM += (om * other.allocation * other.centralize)
    branches[branch].sal += (sal * other.allocation * other.centralize)
    branches[branch].fte += (fte * other.allocation * other.centralize)
    branches[branch].om += (om * other.allocation * other.centralize)




In [33]:
for step in dmi_data.values():
    print(step.totals())

print(f'''Total;  {sum(x.iop_fte for x in dmi_data.values()):,.0f}; {sum(x.iop_personnel for x in dmi_data.values()):,.0f}; {sum(x.iop_OandM for x in dmi_data.values()):,.0f} ''')
print(f'''total value of data related activties:; {iop_totals.fte:,.0f}; {iop_totals.sal:,.0f};{iop_totals.om:,.0f}''')

for branch in branches:
    x = branches[branch]
    print(f'''{branch};{x.fte:,.0f};{x.sal:,.0f};{x.om:,.0f}''')


print(sum(branch.fte for branch in branches.values()) - sum(x.iop_fte for x in dmi_data.values()))
print(sum(branch.sal for branch in branches.values()) - sum(x.iop_personnel for x in dmi_data.values()))
print(sum(branch.om for branch in branches.values()) - sum(x.iop_OandM for x in dmi_data.values()))

Collection;6; 569,827; 346,947 
Receipt;28; 2,885,502; 1,838,403 
Management;43; 4,202,178; 2,964,439 
Cleaning;40; 4,061,348; 2,737,918 
Analysis;48; 4,764,863; 3,169,653 
Dissemination;39; 3,804,222; 2,123,590 
Other;51; 5,054,382; 3,159,018 
Total;  255; 25,342,322; 16,339,969 
total value of data related activties:; 986; 97,970,536;61,232,160
EMB;21;1,939,061;447,982
HPCDPB;46;4,561,913;4,929,533
IDVPB;65;7,046,399;4,141,988
NMLB;118;11,385,792;6,743,216
HSROB;4;347,159;43,350
SPB;1;61,997;33,899
0.0
0.0
0.0


In [128]:
f"{sum(x[1].fte for x in branches.items()):,.0f}"

'250'